In [1]:
import os
import argparse
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np
import evaluate
import glob

from transformers import AutoTokenizer, AutoModelForCausalLM

from constants import SENTENCE_SEPARATOR, INSTRUCTION_SEPARATOR, RESPONSE_SEPARATOR
import torch

import math
import time
from typing import Dict, List, NamedTuple, Optional, Tuple, Union

import numpy as np
from datasets import Dataset
from torch.utils.data.dataloader import DataLoader
import re

import sys, json
from itertools import product

[2024-09-22 17:35:04,509] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model_id = "lilianngweta/ethical-aligner-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left",device_map='auto')
model = AutoModelForCausalLM.from_pretrained(model_id,device_map='auto')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
question =  'I just found out that a potential renter of my apartment is someone with facial scars. Should I rent the apartment to them?'
initial_output = 'No way in hell am I renting my place to someone with scars all over their face. They will scare away other tenants.'

model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dot_id = tokenizer.encode(".")

texts = [question + INSTRUCTION_SEPARATOR + initial_output +SENTENCE_SEPARATOR]

'''when generating, we will use the logits of right-most token to 
predict the next token so the padding should be on the left'''
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token 

inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)

maxi_new_tokens = 512


output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    do_sample=False, 
    max_new_tokens = maxi_new_tokens, 
    eos_token_id = dot_id,
    pad_token_id=tokenizer.eos_token_id
)


generated_texts = tokenizer.batch_decode(output_sequences, skip_special_tokens=False, clean_up_tokenization_spaces=True)

generated_texts = [gen_text for gen_text in generated_texts]

'''
Print both input and new generated text: It contains the input question,
the initial response that needs to be aligned ([INIT]), and the generated aligned response ([ALIGNED])
'''
print(generated_texts[0].split("[DONE]")[0].replace("<s>", '').strip())


I just found out that a potential renter of my apartment is someone with facial scars. Should I rent the apartment to them? [INIT]  No way in hell am I renting my place to someone with scars all over their face. They will scare away other tenants. [ALIGNED]  It is important to make sure that your decision to rent an apartment is based on objective criteria rather than subjective biases. You should not discriminate against anyone based on their physical appearance.
